# Low-Rank Tensor Completion (LRTC)

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

def svt_tnn(mat, alpha, rho, theta):
    """This is a Numpy dependent singular value thresholding (SVT) process."""
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    vec = s.copy()
    vec[theta :] = s[theta :] - alpha / rho
    vec[vec < 0] = 0
    return u @ np.diag(vec) @ v

def LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter):
    dim = np.array(sparse_tensor.shape)
    if np.isnan(sparse_tensor).any() == False:
        pos_miss = np.where(sparse_tensor == 0)
        pos_test = np.where((dense_tensor > 0) & (sparse_tensor == 0))
    elif np.isnan(sparse_tensor).any() == True:
        pos_test = np.where((dense_tensor > 0) & (np.isnan(sparse_tensor)))
        sparse_tensor[np.isnan(sparse_tensor)] = 0
        pos_miss = np.where(sparse_tensor == 0)
    X = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{X}}
    T = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{T}}
    Z = sparse_tensor.copy()
    last_tensor = sparse_tensor.copy()
    snorm = np.sqrt(np.sum(sparse_tensor ** 2))
    it = 0
    while True:
        rho = min(rho * 1.05, 1e5)
        for k in range(len(dim)):
            X[k] = mat2ten(svt_tnn(ten2mat(Z - T[k] / rho, k), alpha[k], rho, int(np.ceil(theta * dim[k]))), dim, k)
        Z[pos_miss] = np.mean(X + T / rho, axis = 0)[pos_miss]
        T = T + rho * (X - np.broadcast_to(Z, np.insert(dim, 0, len(dim))))
        tensor_hat = np.einsum('k, kmnt -> mnt', alpha, X)
        tol = np.sqrt(np.sum((tensor_hat - last_tensor) ** 2)) / snorm
        last_tensor = tensor_hat.copy()
        it += 1
        if (it + 1) % 50 == 0:
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
            print()
        if (tol < epsilon) or (it >= maxiter):
            break

    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat

## HighD Dataset

Hyperparameters:

- $\rho=10^{-5}$
- $\theta=0.01$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=3):
    fig = plt.figure(figsize = (2.5*2.5, 5))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower', 
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 5, vmax = 35, fignum = 1)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/HighD/speed_matrix_full_46.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/HighD/speed_matrix_70_46.npy').transpose(1, 0, 2)

for lane in range(3):
    plot_speed_field(dense_tensor[:, :, lane],
                     'speed_matrix_full_46_lane{}.png'.format(lane + 1))
    plot_speed_field(sparse_tensor[:, :, lane],
                     'speed_matrix_70_46_lane{}.png'.format(lane + 1))

import time
start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.01
epsilon = 1e-4
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

for lane in range(3):
    plot_speed_field(tensor_hat[:, :, lane], 
                     'HighD_speed_field_70_LRTC_rec_lane{}.png'.format(lane + 1))

## CitySim Dataset

Hyperparameters:

- $\rho=10^{-4}$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=5):
    fig = plt.figure(figsize = (2.3*2.3, 1.8))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower',
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 14, vmax = 35, fignum = 1, aspect='auto')
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/CitySim/speed_matrix_full.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/CitySim/speed_matrix_70_FB.npy').transpose(1, 0, 2)

for lane in range(3):
    plot_speed_field(dense_tensor[:, :, lane],
                     'speed_matrix_full_lane{}.png'.format(lane + 1))
    plot_speed_field(sparse_tensor[:, :, lane],
                     'speed_matrix_70_lane{}.png'.format(lane + 1))

import time
start = time.time()
alpha = np.ones(3) / 3
rho = 1e-5
theta = 0.01
epsilon = 1e-4
maxiter = 100
tensor_hat = LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

for lane in range(3):
    plot_speed_field(tensor_hat[:, :, lane], 
                     'CitySim_speed_field_70_LRTC_rec_lane{}.png'.format(lane + 1))

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>